In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from IPython.display import clear_output
from pydrake.all import (
    AbstractValue,
    AddMultibodyPlantSceneGraph,
    Concatenate,
    DiagramBuilder,
    JointSliders,
    LeafSystem,
    MeshcatPoseSliders,
    MeshcatVisualizer,
    MeshcatVisualizerParams,
    Parser,
    PointCloud,
    RandomGenerator,
    Rgba,
    RigidTransform,
    RotationMatrix,
    Simulator,
    StartMeshcat,
    UniformlyRandomRotationMatrix,
)
from manipulation.scenarios import AddMultibodyTriad

from manipulation import running_as_notebook
from manipulation.scenarios import AddFloatingRpyJoint, AddRgbdSensors, ycb
from manipulation.utils import ConfigureParser
import sponana.utils

In [3]:
from sponana.grasping.grasp_generator import get_unified_point_cloud, BananaSystem, ScoreSystem, GenerateAntipodalGraspCandidate

In [4]:
# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7001


In [8]:
def make_internal_model():
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
    parser = Parser(plant)
    sponana.utils.configure_parser(parser)
    parser.AddModelsFromUrl("package://sponana/grasping/banana_and_spot_gripper.dmd.yaml")
    plant.Finalize()
    return builder.Build()


In [9]:
# For visualization
def draw_grasp_candidate(X_G, meshcat, gripper_name, prefix="gripper", draw_frames=True):
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
    parser = Parser(plant)
    sponana.utils.configure_parser(parser)
    parser.AddModelsFromUrl(
        "package://sponana/grasping/spot_gripper.urdf"
    )
    plant.WeldFrames(plant.world_frame(), plant.GetFrameByName(gripper_name), X_G)
    plant.Finalize()

    # frames_to_draw = {"gripper": {"body"}} if draw_frames else {}
    params = MeshcatVisualizerParams()
    params.prefix = prefix
    params.delete_prefix_on_initialization_event = False
    visualizer = MeshcatVisualizer.AddToBuilder(
        builder, scene_graph, meshcat, params
    )
    diagram = builder.Build()
    context = diagram.CreateDefaultContext()
    diagram.ForcedPublish(context)

In [135]:
def sample_grasps_example(gripper_name="arm_link_wr1", pointcloud_transform=RigidTransform()):
    meshcat.Delete()
    rng = np.random.default_rng()

    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
    parser = Parser(plant)
    sponana.utils.configure_parser(parser)
    
    parser.AddModelsFromUrl("package://sponana/grasping/banana_and_spot.dmd.yaml")

    plant.Finalize()

    params = MeshcatVisualizerParams()
    params.prefix = "planning"
    visualizer = MeshcatVisualizer.AddToBuilder(
        builder, scene_graph, meshcat, params
    )
    diagram = builder.Build()
    context = diagram.CreateDefaultContext()
    diagram.ForcedPublish(context)

    # Hide the planning gripper
    meshcat.SetProperty("planning/gripper", "visible", False)

    environment = BananaSystem()
    environment_context = environment.CreateDefaultContext()
    cloud = get_unified_point_cloud(
        environment,
        environment_context,
        meshcat=None
    )
    cloud.mutable_xyzs()[:] = pointcloud_transform.multiply(cloud.xyzs())
    meshcat.SetObject("planning/cloud", cloud, point_size=0.003)

    plant.GetMyContextFromRoot(context)
    scene_graph.GetMyContextFromRoot(context)

    internal_model = make_internal_model()
    internal_model_context = internal_model.CreateDefaultContext()
    costs = []
    X_Gs = []
    for i in range(1000 if running_as_notebook else 2):
        cost, X_G = GenerateAntipodalGraspCandidate(
            internal_model, internal_model_context, cloud, rng,
            wsg_body_index=internal_model.GetSubsystemByName("plant").GetBodyByName(gripper_name).index(),
        )
        if np.isfinite(cost):
            costs.append(cost)
            X_Gs.append(X_G)

    indices = np.asarray(costs).argsort()[:1]
    for rank, index in enumerate(indices):
        draw_grasp_candidate(
            X_Gs[index], meshcat, gripper_name, prefix=f"{rank}th best", draw_frames=False
        )

    print(max(costs))

    return np.array(X_Gs)[indices], plant, context

X_Gs, plant, context = sample_grasps_example(gripper_name="gripper", pointcloud_transform=RigidTransform(
    np.array([0., 0., 0.5])
))

-0.30549159425042627


In [136]:
pcontext = plant.GetMyContextFromRoot(context)
idx = plant.GetBodyByName("body").index()
body_pose = plant.get_body_poses_output_port().Eval(pcontext)[idx]
body_pose

RigidTransform(
  R=RotationMatrix([
    [-0.9999987317275395, -0.0015926529164868282, 0.0],
    [0.0015926529164868282, -0.9999987317275395, 0.0],
    [0.0, 0.0, 1.0],
  ]),
  p=[1.0, 0.0, 0.52],
)

In [151]:
INIT_SPOT_Q = np.array([ 0.7  ,  -0.15  ,  3.14,  0.  , -3.1 ,  3.1 ,  0.  ,  0.  ,  0.  ,  0.  ])

In [152]:
from sponana.controller.inverse_kinematics import solve_ik
from pydrake.all import RollPitchYaw

X_WG = X_Gs[0]
X_AG = RigidTransform(
    RollPitchYaw([0, 0, 1.57]), np.array([0.0, 0.08, 0.00])
)
X_WA = X_WG @ X_AG
soln = solve_ik(plant, plant.GetMyContextFromRoot(context), X_WA, fix_base=True, base_position=INIT_SPOT_Q[:3], position_bound=0.01, rotation_bound=0.01)

In [153]:
soln[-1] = -1.4#-np.pi/2
soln

array([ 0.7       , -0.15      ,  3.14      , -0.31513548, -1.01375138,
        2.18419623,  1.20761062, -0.16630683,  1.27158886, -1.4       ])

In [154]:
meshcat.Delete()
rng = np.random.default_rng()

builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
parser = Parser(plant)
sponana.utils.configure_parser(parser)

parser.AddModelsFromUrl("package://sponana/grasping/banana_and_spot.dmd.yaml")

plant.Finalize()

params = MeshcatVisualizerParams()
params.prefix = "planning"
visualizer = MeshcatVisualizer.AddToBuilder(
    builder, scene_graph, meshcat, params
)
diagram = builder.Build()
context = diagram.CreateDefaultContext()

plant.SetPositions(plant.GetMyContextFromRoot(context), soln)
diagram.ForcedPublish(context)

# Hide the planning gripper
meshcat.SetProperty("planning/gripper", "visible", False)

environment = BananaSystem()
environment_context = environment.CreateDefaultContext()
cloud = get_unified_point_cloud(
    environment,
    environment_context,
    meshcat=None
)
meshcat.SetObject("planning/cloud", cloud, point_size=0.003)

draw_grasp_candidate(
    X_WG, meshcat, "gripper", draw_frames=False
)

## Scratch below here

In [28]:
import numpy as np
from pydrake.all import (
    Context,
    InverseKinematics,
    MultibodyPlant,
    RigidTransform,
    RotationMatrix,
    Solve,
    eq,
)


In [30]:
target_frame_name = "gripper"
base_position = np.zeros(3)
q_nominal_arm = np.array([0.0, -3.1, 3.1, 0.0, 0.0, 0.0, 0.0])
X_WT = X_Gs[0]
position_bound: float = 0.01
rotation_bound: float = 0.01
fix_base = False
pcontext = plant.GetMyContextFromRoot(context)

ik = InverseKinematics(plant, pcontext)
q = ik.q()  # Get variables for MathematicalProgram
prog = ik.prog()  # Get MathematicalProgram

world_frame = plant.world_frame()
target_frame = plant.GetFrameByName(target_frame_name)

# nominal pose
q0 = np.zeros(len(q))
q0[:3] = base_position
q0[3:10] = q_nominal_arm

# Target position and rotation
p_WT = X_WT.translation()
R_WT = X_WT.rotation()

In [35]:
# Constraints
ik.AddPositionConstraint(
    frameA=world_frame,
    frameB=target_frame,
    p_BQ=np.zeros(3),
    p_AQ_lower=p_WT - position_bound,
    p_AQ_upper=p_WT + position_bound,
)
ik.AddOrientationConstraint(
    frameAbar=world_frame,
    R_AbarA=R_WT,
    frameBbar=target_frame,
    R_BbarB=RotationMatrix(),
    theta_bound=rotation_bound,
)
# # This is currently failing for some reason
# # collision constraint
# ik.AddMinimumDistanceLowerBoundConstraint(
#     collision_bound, influence_distance_offset=0.1
# )

# if fix_base:
#     prog.AddConstraint(eq(q[:3], base_position))

# Let's get started
prog.AddQuadraticErrorCost(np.identity(len(q)), q0, q)
prog.SetInitialGuess(q, q0)

result = Solve(ik.prog())
assert result.is_success(), "IK failed :("
return result.GetSolution(q)


AssertionError: IK failed :(